## Tuning Page + Backend

In [1]:
import ipyvuetify as v
from seeq_pid.tuning_page import TuningPage
from seeq_pid.pidtuner import PIDTuner

In [2]:
class Tuning(TuningPage):
    def __init__(self):
        super().__init__()
        
        self.pid = PIDTuner()
        
        self.pid.k_p = 2
        self.pid.tau_p = 30
        self.pid.theta_p = 0
        
        self.pid.set_dt(dt=2)
        self.pid.set_sim_time(200)
        self.pid.delta_max = 0.05
        
        self.pid.setpoint = float(self.sim_card.sp_textbox.v_model)
        
        self.pid.k_c, self.pid.tau_i, self.pid.tau_d, _ = self.pid.rule_based(method='IMC', mode='moderate', tau_c=None)
        
        self.pid_card.p_slider.set_limits(min(0.1*self.pid.k_c, 1), 10*self.pid.k_c, self.pid.k_c)
        self.pid_card.i_slider.set_limits(0.5*self.pid.tau_i, 1.5*self.pid.tau_i, self.pid.tau_i)
        self.pid_card.d_slider.set_limits(0.5*self.pid.tau_d, max(1.5*self.pid.tau_d, self.pid.tau_d+5), self.pid.tau_d)
        
        self.model_card.g_slider.set_limits(0.5*self.pid.k_p, 1.5*self.pid.k_p, self.pid.k_p)
        self.model_card.tau_slider.set_limits(0.5*self.pid.tau_p, 1.5*self.pid.tau_p, self.pid.tau_p)
        self.model_card.d_slider.set_limits(0, self.pid.theta_p+5, self.pid.theta_p)
        
        self.sim_card.sim_time_textbox.v_model = self.pid.sim_time
        
        
        self.pid_card.p_slider.slider.on_event('change', self.slider_action)
        self.pid_card.i_slider.slider.on_event('change', self.slider_action)
        self.pid_card.d_slider.slider.on_event('change', self.slider_action)
        
        self.model_card.g_slider.slider.on_event('change', self.slider_action)
        self.model_card.tau_slider.slider.on_event('change', self.slider_action)
        self.model_card.d_slider.slider.on_event('change', self.slider_action)
        
        self.pid_card.p_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        self.pid_card.i_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        self.pid_card.d_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        
        self.model_card.g_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        self.model_card.tau_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        self.model_card.d_slider.slider_textfield.on_event('change', self.slider_textfiled_action)
        
        self.method_card.method_radio
        
        self.sim_card.sim_time_textbox.on_event('change', self.update_params)
        self.sim_card.sp_textbox.on_event('change', self.update_params)
        
        self.run_pid(mode='optimization')
    
    def slider_action(self, *args):
        self.method_card.method_radio.v_model = 'Custom'
        self.method_card.method_radio_action(item=self.method_card.method_radio)
        self.update_params()
        
    def slider_textfiled_action(self, item, *args):
        cond = item.kwargs['slider_check'](item.v_model)   
        if cond:
            self.update_params()
        
    def update_params(self, *args):
        self.pid.k_p = self.model_card.g_slider.slider.v_model
        self.pid.tau_p = self.model_card.tau_slider.slider.v_model
        self.pid.theta_p = self.model_card.d_slider.slider.v_model
        
        self.pid.k_c = self.pid_card.p_slider.slider.v_model
        self.pid.tau_i = self.pid_card.i_slider.slider.v_model
        self.pid.tau_d = self.pid_card.d_slider.slider.v_model
        
        self.pid.set_sim_time(self.sim_card.sim_time_textbox.v_model)
        self.pid.setpoint = float(self.sim_card.sp_textbox.v_model)
        
        self.pid.reset_sp()
        self.pid.reset_u()
        self.pid.error = 0
        self.pid.integral_error = 0
        self.pid.derivative_error = 0
        self.pid.op_last = 0
        self.pid.pv_last = 0
        
        self.run_pid()
        
    def slider_check(self, item, *args):
        pass
        
    def run_pid(self, mode='simulation', **kwargs):
        if mode=='simulation':
            t, op, pv, sp = self.pid.simulate()
            
        if mode=='optimization':
            t, op, pv, sp = self.pid.optimization_base()
            
        if mode=='rule-based':
            if self.method_card.cl_tau.v_model == '':
                user_cl_tau = None
            else:
                user_cl_tau = float(self.method_card.cl_tau.v_model)
            self.pid.k_c, self.pid.tau_i, self.pid.tau_d, _ = self.pid.rule_based(method='IMC', mode='moderate', tau_c=user_cl_tau)
            t, op, pv, sp = self.pid.simulate()
        
        self.pv_card.update_plot(t, pv, sp, name1='PV', name2='SP')
        self.op_card.update_plot(t, op, name1='OP')
        

In [3]:
a = Tuning()
a

Tuning(children=[Container(children=[CustomData(children=[CardTitle(children=[Spacer(), 'Tuning Constraints', …

In [ ]:
#### History
#### SP
#### Push
#### GitHub
#### Formula